In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Simple Likelihood

In [24]:
#|default_exp simple_likelihood

In [25]:
#|export
import jax
import jax.numpy as jnp
from   jax import jit, vmap
import genjax
from   genjax import gen, choice_map, vector_choice_map
import matplotlib.pyplot as plt
import numpy as np
import bayes3d
from bayes3d._mkl.utils import *

key = jax.random.PRNGKey(0)

In [26]:
#|export
import genjax._src.generative_functions.distributions.tensorflow_probability as gentfp
import tensorflow_probability.substrates.jax as tfp
tfd = tfp.distributions

uniform = genjax.tfp_uniform

truncnormal = gentfp.TFPDistribution(
    lambda mu, sig, low, high: tfd.TruncatedNormal(mu, sig, low, high));

normal = gentfp.TFPDistribution(
    lambda mu, sig: tfd.Normal(mu, sig));

diagnormal = gentfp.TFPDistribution(
    lambda mus, sigs: tfd.MultivariateNormalDiag(mus, sigs));


mixture_of_diagnormals = gentfp.TFPDistribution(
    lambda ws, mus, sig: tfd.MixtureSameFamily(
        tfd.Categorical(ws),
        tfd.MultivariateNormalDiag(mus, sig * jnp.ones_like(mus))))

mixture_of_normals = gentfp.TFPDistribution(
    lambda ws, mus, sig: tfd.MixtureSameFamily(
        tfd.Categorical(ws),
        tfd.Normal(mus, sig * jnp.ones_like(mus))))


mixture_of_truncnormals = gentfp.TFPDistribution(
    lambda ws, mus, sigs, lows, highs: tfd.MixtureSameFamily(
        tfd.Categorical(ws),
        tfd.TruncatedNormal(mus, sigs, lows, highs)))

In [27]:
#|export
from scipy.stats import truncnorm as scipy_truncnormal

normal_logpdf    = jax.scipy.stats.norm.logpdf
truncnorm_logpdf = jax.scipy.stats.truncnorm.logpdf
truncnorm_pdf    = jax.scipy.stats.truncnorm.pdf


In [42]:
#|export
# TODO: The input Y should be an array only containing range measruements as well. 
#       For this to work we need to have the pixel vectors (the rays through each pixel)

def make_simple_sensor_model(zmax):
    """Returns an simple sensor model marginalized over outliers."""    

    @genjax.drop_arguments
    @genjax.gen
    def _sensor_model(y, sig, outlier):
        
        z_ = jnp.linalg.norm(y)

    
        # NOTE: To compare to baseline one should set: `zmax_ = zmax``
        zmax_ = z_/y[2]*zmax

        inlier_outlier_mix = genjax.tfp_mixture(genjax.tfp_categorical, [truncnormal, genjax.tfp_uniform])
        z = inlier_outlier_mix([jnp.log(1.0-outlier), jnp.log(outlier)], (
                                    (z_, sig, 0.0, zmax_), 
                                    (0.0, zmax_))) @ "measurement"

        z = jnp.clip(z, 0.0, zmax_)

        return z * y/z_

        
    @genjax.gen
    def sensor_model(Y, sig, out):   
        """
        Simplest sensor model that returns a vector of range measurements conditioned on 
        an image, noise level, and outlier probability.
        """
                
        X = genjax.Map(_sensor_model, (0,None,None))(Y[...,:3].reshape(-1,3), sig, out) @ "X"
        X = X.reshape(Y.shape)

        return X

    return sensor_model

In [43]:
model = make_simple_sensor_model(5.0)

Y= jnp.array([
    [
            [0,0,1],
            [0,1,2],
    ],
    [
            [1,1,3],
            [1,0,4]
    ]
])
Y.shape
Y[...,2]

Array([[1, 2],
       [3, 4]], dtype=int32)

In [44]:
key = keysplit(key)
model(Y,0.1,0.1)(key)

Array([[[0.       , 0.       , 0.8403682],
        [0.       , 2.1522202, 4.3044405]],

       [[0.4997248, 0.4997248, 1.4991744],
        [0.997259 , 0.       , 3.989036 ]]], dtype=float32)